Hey Esther! I got the code to work, basically I ran the model trying to predict the 'q_score_tier', which is the score of the question.

Accuracy for the model is 43%, maybe we can try re-running the model by binning the accepted answer duration and try predicting that instead.

**The line of code:** 

<div class="alert alert-block alert-info">
    <b># Resample the training data with the BalancedRandomForestClassifier</b>
<br>
from imblearn.ensemble import BalancedRandomForestClassifier
<br>
brfc = BalancedRandomForestClassifier(n_estimators=100,random_state=1)
<br>
rf = brfc.fit(X_train,y_train)
</div>

**takes so long to run, just a warning!**

## Import dependencies and read csv

I saved the df that nisha and suweatha queried into a csv and imported it into this notebook just so our analysis can be a little cleaner

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier

In [2]:
file_path='stack_overflow.csv'
df=pd.read_csv(file_path)

In [3]:
df.head()

,Unnamed: 0,q_id,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id,q_day,q_hour,q_hour_min,accepted_answer_duration
0,0,67341742,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,67341801,Saturday,0,00:03,0.192894
1,1,67341817,Zero Score (0),Short (0 - 50),Short (0 - 10),40-50,47,<50,4,67341857,Saturday,0,00:17,0.149909
2,2,67341895,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,67341911,Saturday,0,00:32,0.076867
3,3,67341936,Positive Score (>0),Medium (50-100),Medium (10-20),50-16000,243,100-250,1,67341961,Saturday,0,00:41,0.088950
4,4,67341921,Positive Score (>0),Long (100-150),Medium (10-20),40-50,57,50-100,2,67341974,Saturday,0,00:38,0.166569


In [4]:
#merge postive score and negative score in q_score_tier but leave zero score
df['target'] = df['q_score_tier'].apply(lambda x: int(x in ['Positive Score (>0)', 'Negative Score (<0)']))
df['target'].value_counts()

0    116033
1     93405
Name: target, dtype: int64

In [6]:
df=df.dropna()

In [7]:
rf_df=df.drop(['Unnamed: 0','q_id','accepted_answer_id','q_score_tier'], axis=1)
rf_df.head()

,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,q_day,q_hour,q_hour_min,accepted_answer_duration,target
0,Medium (50-100),Medium (10-20),30-40,217,100-250,4,Saturday,0,00:03,0.192894,0
1,Short (0 - 50),Short (0 - 10),40-50,47,<50,4,Saturday,0,00:17,0.149909,0
2,Medium (50-100),Medium (10-20),40-50,423,250-500,2,Saturday,0,00:32,0.076867,1
3,Medium (50-100),Medium (10-20),50-16000,243,100-250,1,Saturday,0,00:41,0.088950,1
4,Long (100-150),Medium (10-20),40-50,57,50-100,2,Saturday,0,00:38,0.166569,1


In [12]:
stats_df=rf_df[['target','accepted_answer_duration']]
stats_df.head()

,target,accepted_answer_duration
0,0,0.192894
1,0,0.149909
2,1,0.076867
3,1,0.088950
4,1,0.166569


In [13]:
rf_df.sort_values(by='accepted_answer_duration',ascending=False)

,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,q_day,q_hour,q_hour_min,accepted_answer_duration,target,accepted_answer_duration_bin
207028,Medium (50-100),Medium (10-20),30-40,98,50-100,2,Thursday,14,14:36,2856.969993,0,>1D
201353,Medium (50-100),Short (0 - 10),50-16000,140,100-250,4,Monday,8,08:18,2856.162988,0,>1D
209338,Long (100-150),Medium (10-20),50-16000,140,100-250,4,Sunday,4,04:49,2848.648400,1,>1D
195545,Medium (50-100),Short (0 - 10),50-16000,228,100-250,5,Sunday,10,10:37,2774.281540,0,>1D
198004,Short (0 - 50),Short (0 - 10),50-16000,818,500-10000,5,Tuesday,12,12:57,2758.764791,1,>1D
...,...,...,...,...,...,...,...,...,...,...,...,...
118903,Medium (50-100),Short (0 - 10),10-20,95,50-100,3,Thursday,23,23:38,0.004629,0,<1H
204995,Short (0 - 50),Short (0 - 10),20-30,69,50-100,3,Wednesday,11,11:27,0.003704,0,<1H
171188,Medium (50-100),Short (0 - 10),30-40,149,100-250,2,Tuesday,16,16:30,0.000194,1,<1H
165801,Medium (50-100),Medium (10-20),10-20,41,<50,2,Friday,19,19:34,0.000191,1,<1H


In [14]:
answer_bins = [0, 1, 24, 3000]
answer_bins_group_names = ["<1H", "<1D", ">1D"]

# Categorize score based on the bins.
rf_df['accepted_answer_duration_bin'] = pd.cut(rf_df['accepted_answer_duration'], answer_bins, labels=answer_bins_group_names)

In [15]:
rf_df.head()

,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,q_day,q_hour,q_hour_min,accepted_answer_duration,target,accepted_answer_duration_bin
0,Medium (50-100),Medium (10-20),30-40,217,100-250,4,Saturday,0,00:03,0.192894,0,<1H
1,Short (0 - 50),Short (0 - 10),40-50,47,<50,4,Saturday,0,00:17,0.149909,0,<1H
2,Medium (50-100),Medium (10-20),40-50,423,250-500,2,Saturday,0,00:32,0.076867,1,<1H
3,Medium (50-100),Medium (10-20),50-16000,243,100-250,1,Saturday,0,00:41,0.088950,1,<1H
4,Long (100-150),Medium (10-20),40-50,57,50-100,2,Saturday,0,00:38,0.166569,1,<1H


In [16]:
stats_df.groupby('target').mean()

,accepted_answer_duration
target,
0,26.289773
1,23.965796


In [17]:
rf_df.groupby('accepted_answer_duration_bin').count()

,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,q_day,q_hour,q_hour_min,accepted_answer_duration,target
accepted_answer_duration_bin,,,,,,,,,,,
<1H,111305,111305,111305,111305,111305,111305,111305,111305,111305,111305,111305
<1D,71404,71404,71404,71404,71404,71404,71404,71404,71404,71404,71404
>1D,26711,26711,26711,26711,26711,26711,26711,26711,26711,26711,26711


## Create features and encode our features using pd.get_dummies

In [19]:
# Create our features
X = rf_df.drop('target', axis=1)
X = pd.get_dummies(X)

# Create our target
y = rf_df["target"]

X.head()

,q_body_word_count,q_tags_count,q_hour,accepted_answer_duration,q_title_char_count_bin_Long (100-150),q_title_char_count_bin_Medium (50-100),q_title_char_count_bin_Short (0 - 50),q_title_word_count_bin_Long (20-30),q_title_word_count_bin_Medium (10-20),q_title_word_count_bin_Short (0 - 10),...,q_hour_min_23:53,q_hour_min_23:54,q_hour_min_23:55,q_hour_min_23:56,q_hour_min_23:57,q_hour_min_23:58,q_hour_min_23:59,accepted_answer_duration_bin_<1H,accepted_answer_duration_bin_<1D,accepted_answer_duration_bin_>1D
0,217,4,0,0.192894,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,47,4,0,0.149909,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,423,2,0,0.076867,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,243,1,0,0.088950,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,57,2,0,0.166569,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
X.describe()

,q_body_word_count,q_tags_count,q_hour,accepted_answer_duration,q_title_char_count_bin_Long (100-150),q_title_char_count_bin_Medium (50-100),q_title_char_count_bin_Short (0 - 50),q_title_word_count_bin_Long (20-30),q_title_word_count_bin_Medium (10-20),q_title_word_count_bin_Short (0 - 10),...,q_hour_min_23:53,q_hour_min_23:54,q_hour_min_23:55,q_hour_min_23:56,q_hour_min_23:57,q_hour_min_23:58,q_hour_min_23:59,accepted_answer_duration_bin_<1H,accepted_answer_duration_bin_<1D,accepted_answer_duration_bin_>1D
count,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,...,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000,209420.000000
mean,183.645010,2.987776,12.306676,25.253349,0.054804,0.558318,0.386878,0.020628,0.362496,0.616679,...,0.000411,0.000382,0.000363,0.000425,0.000401,0.000463,0.000415,0.531492,0.340961,0.127548
std,173.890508,1.230564,5.891066,119.035545,0.227597,0.496589,0.487037,0.142137,0.480722,0.486197,...,0.020261,0.019541,0.019047,0.020611,0.020024,0.021517,0.020378,0.499008,0.474033,0.333586
min,5.000000,1.000000,0.000000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,88.000000,2.000000,8.000000,0.237724,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,139.000000,3.000000,13.000000,0.832657,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,222.000000,4.000000,17.000000,5.658932,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,9288.000000,5.000000,23.000000,2856.969993,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
# Check the balance of our target values
y.value_counts()

0    116025
1     93395
Name: target, dtype: int64

## Split data to training and testing sets

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)

## Fit model: Random Forest Classifier

In [23]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brfc = BalancedRandomForestClassifier(n_estimators=100,random_state=1)
rf = brfc.fit(X_train,y_train)

## Calculate Accuracy

In [24]:
# Calculated the balanced accuracy score
y_pred=rf.predict(X_test)
ba_balanced_forest=balanced_accuracy_score(y_test,y_pred)
ba_balanced_forest

0.5697255370467214

Accuracy of the model is **57%**

## Display Confusion Matrix


### Confusion Matrix with 3 Categories

(this is a little confusing, we should either bin it into 2 categories for ease of analysis or use the expected answer duration as our y value and bin that into 2 categories)

|                           | Predicted Negative Score (<0)  |Predicted Zero Score (0)|Predicted Positive Score (>0)|
| --------------------------| ------------------------------ |------------------------|-----------------------------|
| Actual Negative Score (<0)|    TRUE POSITIVE               |    FALSE NEGATIVE      | FALSE NEGATIVE              |
| Actual Zero Score (0)     |                                |     TRUE POSITIVE      | TRUE POSITIVE
|Actual Positive Score (>0)	|

In [31]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

cm_df=pd.DataFrame(cm,
                  index=[ "Actual Zero Score (0)", "Actual Positive Score (>0)"],
                  columns=["Positive Score (>0)", "Negative Score (<0)"],
cm_df

SyntaxError: unexpected EOF while parsing (<ipython-input-31-befeaf5faa33>, line 8)

## Print additional scores for analysis: precision, recall, and f1

In [32]:
# Print the imbalanced classification report
icr_balanced_forest=classification_report_imbalanced(y_test,y_pred)
print(icr_balanced_forest)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.63      0.56      0.58      0.59      0.57      0.33     29006
          1       0.52      0.58      0.56      0.55      0.57      0.33     23349

avg / total       0.58      0.57      0.57      0.57      0.57      0.33     52355



In [33]:
#Summary of findings

print(f'For the Balanced Random Forest Classifier algortihm, the balanced accuracy score is {ba_balanced_forest}' 
      f'\n\nand the imbalanced classifcation report is:\n\n{icr_balanced_forest}')

For the Balanced Random Forest Classifier algortihm, the balanced accuracy score is 0.5724542704943864

and the imbalanced classifcation report is:

                   pre       rec       spe        f1       geo       iba       sup

          0       0.63      0.56      0.58      0.59      0.57      0.33     29006
          1       0.52      0.58      0.56      0.55      0.57      0.33     23349

avg / total       0.58      0.57      0.57      0.57      0.57      0.33     52355



## Feature Importance

In [34]:
# List the features sorted in descending order by feature importance
sorted(zip(rf.feature_importances_, X.columns), reverse=True)

[(0.0989156611319299, 'accepted_answer_duration'),
 (0.09194490177359749, 'Unnamed: 0'),
 (0.08512278258673041, 'q_body_word_count'),
 (0.056490718117037764, 'q_hour'),
 (0.036221655919965394, 'q_tags_count'),
 (0.012778232620100898, 'q_view_count_bin_50-16000'),
 (0.009722980234117197, 'q_title_char_count_bin_Medium (50-100)'),
 (0.008782488867584863, 'q_day_Thursday'),
 (0.008766097527868616, 'q_day_Wednesday'),
 (0.00874776150620463, 'q_day_Tuesday'),
 (0.008532027599733528, 'q_title_char_count_bin_Short (0 - 50)'),
 (0.008334767524378473, 'q_title_word_count_bin_Medium (10-20)'),
 (0.00831066590135145, 'q_title_word_count_bin_Short (0 - 10)'),
 (0.008184438782825796, 'q_day_Friday'),
 (0.007953155737254863, 'q_day_Monday'),
 (0.007663666424002121, 'q_view_count_bin_10-20'),
 (0.007405008831354618, 'q_view_count_bin_20-30'),
 (0.0071350695835999955, 'q_body_len_bin_100-250'),
 (0.006660085977632191, 'q_day_Saturday'),
 (0.005844952513064322, 'q_body_len_bin_50-100'),
 (0.00555416950

Takeaway/Next Steps:
    
    - add more features/expand data to try and improve accuracy
    - conclude there is no real correlation between features and output, look at the data from a different perspective
        - ie. cluster unanswered questions?